In [ ]:
from pathlib import Path

In [ ]:
# optional, enable logging
import logging
logging.basicConfig(level='INFO')

In [ ]:
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# first, we create a list of paths with all files we want to process
paths = [
    'ISIMIP3b/OutputData/biomes/CLASSIC/gfdl-esm4/future/classic_gfdl-esm4_w5e5_ssp585_2015soc-from-histsoc_default_burntarea-total_global_monthly_2015_2100.nc'
]

In [ ]:
# then, we start a job to select a bounding box and poll every 4 seconds
response = client.select_bbox(paths, -20, 40, 0, 20, poll=4)

In [ ]:
# when the job is done, the result can be downloaded and extracted
if 'file_url' in response:
    client.download(response['file_url'], path='downloads', extract=True)

In [ ]:
# the result can be plotted using xarray and matplotlib
import matplotlib.pyplot as plt
import xarray as xr

path = 'downloads/classic_gfdl-esm4_w5e5_ssp585_2015soc-from-histsoc_default_burntarea-total_lon-20.0to40.0lat0.0to20.0_monthly_2015_2100.nc'
ds = xr.open_dataset(path)
ds['burntarea-total'].isel(time=0).plot()

In [ ]:
# the following methods can be used in a similar way
client.select_point(paths, lat, lon, poll=poll)
client.mask_bbox(paths, west, east, south, north, poll=poll)
client.mask_country(paths, country, poll=poll)
client.mask_landonly(paths, poll=poll)
client.mask_mask(paths, mask, var, poll=poll)
client.mask_shape(paths, shapefile, layer, poll=poll)
client.mask_shape(paths, geojson, layer, poll=poll)
client.cutout_bbox(paths, west, east, south, north, poll=poll)
client.cutout_point(paths, lat, lon, poll=poll)

In [ ]:
# in addition, the client allows to use the API with a custom list of operations
# this can be used to create a cut-out of a country from high resolution data, e.g.
paths = [
    'ISIMIP3a/InputData/climate/atmosphere/obsclim/global/daily/historical/CHELSA-W5E5/chelsa-w5e5_obsclim_tas_30arcsec_global_daily_201612.nc',
]

In [ ]:
# we use the Admin 0 – Countries data from naturalearthdata.com, which we need to download
# first from https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
ne_path = Path('downloads') / 'ne_10m_admin_0_countries.zip'
ne_mask = ne_path.with_suffix('.nc')

In [ ]:
# we now need to prepare a chain of three operations
operations = [
    {
        'operation': 'cutout_bbox',
        'bbox': [
             5.800,  # west
            10.600,  # east
            45.800,  # south
            47.900   # north
        ]
    },
    {
        'operation': 'create_mask',
        'shape': ne_path.name,
        'mask': ne_mask.name,
    },
    {
        'operation': 'mask_mask',
        'mask': ne_mask.name,
        'var': 'm_91'  # switzerland layer 91 in the shapefile
    }
]

In [ ]:
# we also need a list of files to upload, which are referenced in the operations list
uploads = [ne_path]

In [ ]:
# finally, we can submit the prepared job to the API and poll every 4 seconds for it's status
response = client.submit_job(paths, operations, uploads, poll=4)

In [ ]:
# when the job is done, the result can be downloaded and extracted
if 'file_url' in response:
    client.download(response['file_url'], path='downloads', extract=True)

In [ ]:
# again, the result can be plotted using xarray and matplotlib
import matplotlib.pyplot as plt
import xarray as xr

path = 'downloads/chelsa-w5e5_obsclim_tas_30arcsec_lon5.8to10.6lat45.8to47.9+ne_10m_admin_0_countries_daily_201612.nc'
ds = xr.open_dataset(path)
ds['tas'].isel(time=10).plot()